In [ ]:
import yfinance as yf
import panda as pd
import matplotlib.pyplot as plt
#download spy data
spy = yf.download( 'SPY', Start='2018-01-01', end='2023-01-01')

#calculate 20 days and 50 days moving average
spy['SMA20'] = spy ['close'].rolling(window=20).mean()
spy['SMA50'] = spy['Close'].rolling(window=50).mean()

#plot
plt.figure(figsize=(12,16))
plt.plot(spy['close'], label='spy close')
plt.plot(spy['SMA20'], label='SMA20')
plt.plot(spy['SMA50'], label='SMA50')
plt.title("spy with 20/50 SMA crossover")
plt.legend()
plt.show()